In [2]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader, Dataset
import matplotlib.pyplot as plt

In [3]:
import torch.nn as nn
import torch 
import torch.nn.functional as F

def init_weights(m):
    if isinstance(m, nn.Embedding):
        torch.nn.init.normal_(m.weight.data, mean=0.0, std=0.01)
    elif isinstance(m, nn.Linear):
        torch.nn.init.normal_(m.weight.data, mean=0.0, std=0.01)
        if m.bias is not None:
            torch.nn.init.constant_(m.bias.data, 0)

class GMF(nn.Module):
    def __init__(self, num_users, num_items, embedding_size):
        super(GMF, self).__init__()
        self.user_embedding = nn.Embedding(num_users, embedding_size)
        self.item_embedding = nn.Embedding(num_items, embedding_size)
        self.fc = nn.Linear(embedding_size, 1)
        
    def forward(self, user_ids, item_ids):
        user_embedding = self.user_embedding(user_ids)
        item_embedding = self.item_embedding(item_ids)
        element_product = user_embedding * item_embedding
        logits = self.fc(element_product)
        return logits.view(-1)


class MLP(nn.Module):
    def __init__(self, num_users, num_items, hidden_sizes):
        super(MLP, self).__init__()
        self.num_users = num_users
        self.num_items = num_items
        
        layers = []
        input_size = num_users + num_items
        layer_sizes = [input_size] + hidden_sizes + [1] 
        for i in range(len(layer_sizes) - 1):
            layers.append(nn.Linear(layer_sizes[i], layer_sizes[i+1]))
            if i < len(layer_sizes) - 2:
                layers.append(nn.ReLU())  
        self.mlp = nn.Sequential(*layers)

    def forward(self, user_ids, item_ids):
        user_onehot = F.one_hot(user_ids, num_classes=self.num_users).float()
        item_onehot = F.one_hot(item_ids, num_classes=self.num_items).float()
        input_vec = torch.cat([user_onehot, item_onehot], dim=1)
        return self.mlp(input_vec)

class NeuMF(nn.Module):
    def __init__(self, num_users, num_items, mf_dim=10, mlp_layers=[64, 32], dropout=0.2):
        super(NeuMF, self).__init__()
        self.user_embedding_mf = nn.Embedding(num_users, mf_dim)
        self.item_embedding_mf = nn.Embedding(num_items, mf_dim)
        layers = []
        input_size = mf_dim * 2
        layer_sizes = [input_size] + mlp_layers
        for i in range(len(layer_sizes) - 1):
            layers.append(nn.Linear(layer_sizes[i], layer_sizes[i+1]))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(dropout))
        self.mlp = nn.Sequential(*layers)
        self.output_layer = nn.Linear(mlp_layers[-1] + mf_dim, 1)

    def forward(self, user_ids, item_ids):
        user_embedding_mf = self.user_embedding_mf(user_ids)
        item_embedding_mf = self.item_embedding_mf(item_ids)
        mf_output = torch.mul(user_embedding_mf, item_embedding_mf)  # Element-wise multiplication

        user_embedding_mlp = self.user_embedding_mf(user_ids)
        item_embedding_mlp = self.item_embedding_mf(item_ids)
        mlp_input = torch.cat((user_embedding_mlp, item_embedding_mlp), dim=1)
        mlp_output = self.mlp(mlp_input)

        concat_output = torch.cat((mf_output, mlp_output), dim=1)
        output = self.output_layer(concat_output).squeeze()
        return output

In [4]:
import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import numpy as np
import pandas as pd

# from model import GMF, MLP, NeuMF
# from evaluation import evaluate_model
import re 
from torch.utils.data import TensorDataset, DataLoader

from torch.utils.data import Dataset

learning_rate = 0.001
epochs = 100
data_dir = 'ml-1m'
batch_size = 64


class NegativeDataset(Dataset):
    def __init__(self, negative_file):
        self.negatives = self._load_negative(negative_file)

    def _load_negative(self, negative_file):
        negatives = {}
        with open(negative_file, 'r') as file:
            for line in file:
                line = line.strip().split('\t')
                user, item = eval(line[0])
                negatives[(user, item)] = list(map(int, line[1:]))
        return negatives

    def __len__(self):
        return len(self.negatives)

    def __getitem__(self, idx):
        user_item = list(self.negatives.keys())[idx]
        user_id = torch.tensor(user_item[0], dtype=torch.int64)
        item_id = torch.tensor(user_item[1], dtype=torch.int64)
        negative_ids = torch.tensor(self.negatives[user_item], dtype=torch.int64)
        return user_id, item_id, negative_ids


train_data = pd.read_csv(f'{data_dir}/ml-1m.train.rating', sep='\t', header=None, names=['user', 'item', 'rating', 'timestamp'])
train_data = train_data.drop(columns='timestamp')
test_data = pd.read_csv(f'{data_dir}/ml-1m.test.rating', sep='\t', header=None, names=['user', 'item', 'rating', 'timestamp'])
test_data = test_data.drop(columns='timestamp')
negative_dataset = NegativeDataset('./ml-1m/ml-1m.test.negative')
negative_loader = DataLoader(negative_dataset, batch_size=batch_size, shuffle=False)
train_dataset = TensorDataset(torch.tensor(train_data['user'].values),
                          torch.tensor(train_data['item'].values),
                          torch.tensor(train_data['rating'].values))
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
test_dataset = TensorDataset(torch.tensor(test_data['user'].values),
                          torch.tensor(test_data['item'].values),
                          torch.tensor(test_data['rating'].values))
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
test_users, test_items = torch.tensor(test_data['user'].values), torch.tensor(test_data['item'].values)

In [5]:
num_users = max(train_data['user'].max(), test_data['user'].max()) + 1
print(num_users)
num_items = max(train_data['item'].max(), test_data['item'].max()) + 1
print(num_items)

embedding_size = 64
gmf_model = GMF(num_users, num_items, embedding_size)
hidden_sizes = [16]  
mlp_model_1 = MLP(num_users, num_items, hidden_sizes)
hidden_sizes = [16, 8] 
mlp_model_2 = MLP(num_users, num_items, hidden_sizes)
hidden_sizes = [16, 64, 8] 
mlp_model_3 = MLP(num_users, num_items, hidden_sizes)
hidden_sizes = [16, 64, 16, 8] 
mlp_model_4 = MLP(num_users, num_items, hidden_sizes)
neumf_model = NeuMF(num_users, num_items, mf_dim=10, mlp_layers=[64, 32], dropout=0.2)

6040
3706


In [6]:
import numpy as np
import torch


def evaluate(model, test_loader, negative_loader, top_k=10):
    model.eval()
    HR_list = []
    NDCG_list = []

    for (user_ids, pos_item_ids, _), (neg_user_ids, _, neg_item_ids) in zip(test_loader, negative_loader):
        items = torch.cat([pos_item_ids.unsqueeze(1), neg_item_ids], dim=1)
        user_ids = user_ids.unsqueeze(1).expand(-1, items.size(1))

        user_ids = user_ids.reshape(-1)
        item_ids = items.reshape(-1)
        predictions = model(user_ids, item_ids).squeeze()
        predictions = predictions.reshape(-1, 100)
        _, indices = torch.topk(predictions, k=top_k, dim=1)
        recommended_items = items.gather(1, indices)

        HR = (recommended_items == pos_item_ids.unsqueeze(1)).any(dim=1).float()
        HR_list.append(HR.mean().item())

        relevant = (recommended_items == pos_item_ids.unsqueeze(1))
        rank = relevant.nonzero(as_tuple=True)[1]
        NDCG = (1 / torch.log2(rank.float() + 2)).mean().item()
        NDCG_list.append(NDCG)

    mean_HR = np.mean(HR_list)
    mean_NDCG = np.mean(NDCG_list)
    
    return mean_HR, mean_NDCG

In [ ]:
import math

results = []
model = gmf_model

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
model.train()
for epoch in range(30):
    for users, items, ratings in train_loader:
        criterion = nn.BCEWithLogitsLoss()
        predictions = model(users, items).squeeze()
        predictions = predictions.view(-1)
        loss = criterion(predictions, ratings.float())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Epoch {epoch+1}/{30}')
    hr, ndcg = evaluate(model, test_loader, negative_loader, top_k=10)
    results.append({
        'Model': 'GMF',
        'HR@10': hr,
        'NDCG@10': ndcg
    })
    print(f"Model: {model.__class__.__name__}, HR@10: {hr}, NDCG@10: {ndcg}")
    
results_df = pd.DataFrame(results)
results_df.to_csv('GMF.csv', index=False)

In [10]:
results = []
count = 0
for model in [mlp_model_1, mlp_model_2, mlp_model_3, mlp_model_4]:
    count = count + 1
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    model.train()
    for epoch in range(30):
        for users, items, ratings in train_loader:
            criterion = nn.BCEWithLogitsLoss()
            predictions = model(users, items).squeeze()
            predictions = predictions.view(-1)
            loss = criterion(predictions, ratings.float())
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
        print(f'Epoch {epoch+1}/{30}')
        hr, ndcg = evaluate(model, test_loader, negative_loader, top_k=10)
        results.append({
            'Model': 'MLP_' + str(count),
            'HR@10': hr,
            'NDCG@10': ndcg
        })
        print(f"Model: MLP_{count}, HR@10: {hr}, NDCG@10: {ndcg}")
        
results_df = pd.DataFrame(results)
results_df.to_csv('MLP.csv', index=False)

In [7]:
results = []
model = neumf_model
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
model.train()
for epoch in range(30):
    for users, items, ratings in train_loader:
        criterion = nn.BCEWithLogitsLoss()
        predictions = model(users, items).squeeze()
        predictions = predictions.view(-1)
        loss = criterion(predictions, ratings.float())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Epoch {epoch+1}/{30}')
    hr, ndcg = evaluate(model, test_loader, negative_loader, top_k=10)
    results.append({
        'Model': model.__class__.__name__,
#             'Loss': loss.item(),
        'HR@10': hr,
        'NDCG@10': ndcg
    })
    print(f"Model: {model.__class__.__name__}, HR@10: {hr}, NDCG@10: {ndcg}")

results_df = pd.DataFrame(results)
results_df.to_csv('NeuMF.csv', index=False)

Epoch 1/30
Model: NeuMF, HR@10: 0.4447916667712362, NDCG@10: 0.5538063309694591
Epoch 2/30
Model: NeuMF, HR@10: 0.4459429825607099, NDCG@10: 0.5587665598643453
Epoch 3/30
Model: NeuMF, HR@10: 0.44544956150807835, NDCG@10: 0.559753398832522
Epoch 4/30
Model: NeuMF, HR@10: 0.4452850878238678, NDCG@10: 0.560041973779076
Epoch 5/30
Model: NeuMF, HR@10: 0.446600877297552, NDCG@10: 0.558251712824169
Epoch 6/30
Model: NeuMF, HR@10: 0.44627192992913095, NDCG@10: 0.5578570930581344
Epoch 7/30
Model: NeuMF, HR@10: 0.4457785088764994, NDCG@10: 0.5586960654509695
Epoch 8/30
Model: NeuMF, HR@10: 0.44610745624492043, NDCG@10: 0.5584603849210237
Epoch 9/30
Model: NeuMF, HR@10: 0.4459429825607099, NDCG@10: 0.5583915290079619
Epoch 10/30
Model: NeuMF, HR@10: 0.4459429825607099, NDCG@10: 0.5577109474884836
Epoch 11/30
Model: NeuMF, HR@10: 0.44643640361334147, NDCG@10: 0.5573945196051346
Epoch 12/30
Model: NeuMF, HR@10: 0.446600877297552, NDCG@10: 0.5573740720748901
Epoch 13/30
Model: NeuMF, HR@10: 0.446